In [2]:
from numpy import array
import numpy as np
from keras.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
from matplotlib import pyplot as plt

In [5]:
# define model

m = int(800)
n = int(800)

model = Sequential()
model.add(LSTM(800, activation='relu', return_sequences=True, input_shape=(m,n)))
model.add(LSTM(800, activation='relu'))
model.add(Dense(3,  activation='relu'))
model.compile(optimizer='adam', loss=mean_squared_error)